# Text to Audio Experiments

In [1]:
import getpass

# Prompt for the OpenAI API key
openai_api_key = getpass.getpass("Please enter your OpenAI API key: ")
print("OpenAI API key captured successfully!")

# Prompt for the Replicate API key
replicate_api_key = getpass.getpass("Please enter your Replicate API key: ")
print("Replicate API key captured successfully!")

# Prompt for the Grok API key
groq_api_key = getpass.getpass("Please enter your Groq API key: ")
print("Groq API key captured successfully!")

OpenAI API key captured successfully!
Replicate API key captured successfully!
Groq API key captured successfully!


## Suno Bark: text to audio

In [2]:
import replicate

# Create a Replicate client instance with the API token
client = replicate.Client(api_token=replicate_api_key)

# Define the input parameters for the model
input_params = {
    "prompt": "Hello, my name is Hugo. And, uh — and I like pizza. [laughs] But I also have other interests such as playing tic tac toe.",
    "text_temp": 0.7,
    "output_full": False,
    "waveform_temp": 0.7,
    "history_prompt": "announcer"
}

# Run the model using Replicate API
try:
    output = client.run(
        "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
        input=input_params
    )
    print(output)
except Exception as e:
    print(f"Error: {e}")

{'audio_out': 'https://replicate.delivery/czjl/YuNTXATojnb4AxMEmJ4MtHQmtI2ctdCkuSbQbEVUFTuVOO0E/audio.wav'}


### LLM output --> Suno bark

In [3]:
from openai import OpenAI
import os


def get_llm_response(user_input):
    client = OpenAI(api_key=openai_api_key)
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
    {"role": "user", "content": user_input}
  ]
        )
    return response.choices[0].message.content

In [4]:
song = get_llm_response("a short pirates sea shanty")
print(song)

Yo ho, me hearties, gather 'round
A tale of pirates on the sea profound
With waves a-crashing, winds a-blowing
Adventure on the high seas, we be going

Chorus:
Sail, me lads, on this ship divine
Raise the anchor, set the sails, in rhythm we'll chime
With hooks for hands and patches for eyes
We pirates sail the seven seas, bold and wise

Oh, the captain be a fearsome man
With a peg leg and a hook, he leads the band
His voice rides the wind, as we set sail
A true scallywag, he never does fail

(Chorus)

We plunder and pillage, seeking treasure untold
From the Caribbean to lands unknown, bold
A chest full of gold, a map to guide the way
With hearts full of mischief, we're off to play

(Chorus)

Through storms and squalls, we'll never back down
For the thrill of the chase, we wear our frown
With cannons ablaze, we conquer the foe
There's no place for weakness in our pirate show

(Chorus)

So raise your rum, me hearties, in toast
To the adventure that lurks along the coast
With hearts as wi

In [5]:
# Define the input parameters for the model
input_params = {
    "prompt": song,
    "text_temp": 0.7,
    "output_full": False,
    "waveform_temp": 0.7,
    "history_prompt": "announcer",
   # "duration": 30
}

# Run the model using Replicate API
try:
    output = client.run(
        "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
        input=input_params
    )
    print(output)
except Exception as e:
    print(f"Error: {e}")

{'audio_out': 'https://replicate.delivery/czjl/Fd8GCWYtW96hGJUa1SmBnKXoO4Q1DyvYawfx2fuMBA2L64QTA/audio.wav'}


This is totally bent and makes no sense lolz ^

## Text to music w/ meta musicgen

In [6]:
input = {
    "prompt": "Horns and Drums. Edo25 major g melodies that sound triumphant and cinematic. Leading up to a crescendo that resolves in a 9th harmonic",
    "model_version": "stereo-large",
    "output_format": "mp3",
    "normalization_strategy": "peak"
}

output = client.run(
    "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
    input=input
)
print(output)
#=> "https://replicate.delivery/pbxt/OeLYIQiltdzMaCex1shlEFy6...

https://replicate.delivery/yhqm/J5PQpE2tNuqMOlQ0f2DMaX2QayhtfpaPLRwv1LBDJqAk64QTA/out.mp3


In [7]:
input = {
    "prompt": "Ancient Trip Hop with Throat Singing",
    "model_version": "stereo-large",
    "output_format": "mp3",
    "normalization_strategy": "peak",
    "duration": 30 
}

output = client.run(
    "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
    input=input
)
print(output)
#=> "https://replicate.delivery/pbxt/OeLYIQiltdzMaCex1shlEFy6...

https://replicate.delivery/yhqm/04DiXRG65647OxukBKSChc7FzJ0eIQ081jzL4fUmParq74QTA/out.mp3


## Text to music with riffusion

In [8]:
output = client.run(
    "riffusion/riffusion:8cf61ea6c56afd61d8f5b9ffd14d7c216c0a93844ce2d82ac1c9ecc9c7f24e05",
    input={
        "alpha": 0.5,
        "prompt_a": "West African Desert Blues",
        "prompt_b": "Throat Singing",
        "denoising": 0.75,
        "seed_image_id": "vibes",
        "num_inference_steps": 50
    }
)
print(output)

{'audio': 'https://replicate.delivery/czjl/7NYgDjSfzHwKaqSH5xI6DtIwcWEbH5Qljco9e8WeOjKo3xhmA/gen_sound.wav', 'spectrogram': 'https://replicate.delivery/czjl/uOVUtpY6JsJvERml80llAwMipfltFj9zh9RbVGkLPUP6dcoJA/spectrogram.jpg'}


___

## Experiment: One prompt to many models

In [9]:
message = "The Waffle House is really messing shit up with pancakes and bacon tonight HOLY MOLEY and there's anarchist jazz also!"

### text to image

In [10]:
input = {
    "prompt": message
}

output = client.run(
    "fofr/epicrealismxl-lightning-hades:0ca10b1fd361c1c5568720736411eaa89d9684415eb61fd36875b4d3c20f605a",
    input=input
)
print(output)
#=> ["https://replicate.delivery/pbxt/ulYZRIyAUDYpOZfl7OjhrKx...

['https://replicate.delivery/pbxt/lEMHeZDieDnX3E8Bx3KTiTeDgkPHi2xA6WdHs6NRDsmq6xhmA/R8__00001_.webp']


### text to audio

In [11]:
# Define the input parameters for the model
input_params = {
    "prompt": message,
    "text_temp": 0.7,
    "output_full": False,
    "waveform_temp": 0.7,
    "history_prompt": "announcer",
   # "duration": 30
}

# Run the model using Replicate API
try:
    output = client.run(
        "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
        input=input_params
    )
    print(output)
except Exception as e:
    print(f"Error: {e}")

{'audio_out': 'https://replicate.delivery/czjl/dzrRuXmL1EYZNxfuRNxINYurL6vXpjaZRR738G5OxS87e4QTA/audio.wav'}


### text to music

In [12]:
input = {
    "prompt": message,
    "model_version": "stereo-large",
    "output_format": "mp3",
    "normalization_strategy": "peak",
    "duration": 30 
}

output = client.run(
    "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
    input=input
)
print(output)
#=> "https://replicate.delivery/pbxt/OeLYIQiltdzMaCex1shlEFy6...

https://replicate.delivery/yhqm/Ium0EE1yDJoALtYICuGsq3eAafq9yLW79MwMHhZbMgL8exhmA/out.mp3


## Many models at once

In [17]:
import replicate

def generate_epic_realism(prompt, api_token):
    # Create a Replicate client instance with the API token
    client = replicate.Client(api_token=replicate_api_key)

    # Define the input parameters for the model
    input_data = {
        "prompt": prompt
    }

    # Run the model using Replicate API
    output = client.run(
        "fofr/epicrealismxl-lightning-hades:0ca10b1fd361c1c5568720736411eaa89d9684415eb61fd36875b4d3c20f605a",
        input=input_data
    )
    
    return output

# Example usage
# api_token = XXX
message = "crazy wild zombie party at the blaring symphony orchestra"
output = generate_epic_realism(message, replicate_api_key)
print(output)


def generate_suno_bark(prompt, api_token, text_temp=0.7, output_full=False, waveform_temp=0.7, history_prompt="announcer"):
    # Create a Replicate client instance with the API token
    client = replicate.Client(api_token=replicate_api_key)

    # Define the input parameters for the model
    input_params = {
        "prompt": prompt,
        "text_temp": text_temp,
        "output_full": output_full,
        "waveform_temp": waveform_temp,
        "history_prompt": "zh_speaker_7",
    }

    # Run the model using Replicate API
    try:
        output = client.run(
            "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
            input=input_params
        )
        return output
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
# api_token = 'your_api_token_here'
# message = "Your prompt here"
output = generate_suno_bark(message, replicate_api_key)
print(output)

import replicate

def generate_music_gen(prompt, api_token, duration=30, model_version="stereo-large", output_format="mp3", normalization_strategy="peak"):
    # Create a Replicate client instance with the API token
    client = replicate.Client(api_token=replicate_api_key)

    # Define the input parameters for the model
    input_data = {
        "prompt": prompt,
        "model_version": model_version,
        "output_format": output_format,
        "normalization_strategy": normalization_strategy,
        "duration": duration 
    }

    # Run the model using Replicate API
    output = client.run(
        "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
        input=input_data
    )
    
    return output

# Example usage
# api_token = 'your_api_token_here'
# message = "Ancient Trip Hop with Throat Singing"
output = generate_music_gen(message, replicate_api_key)
print(output)



['https://replicate.delivery/pbxt/zD6E1bsjewxjQiKhEv2Dt2vCbQ6vzhYbBHLzQTJVA4VxhcoJA/R8__00001_.webp']
{'audio_out': 'https://replicate.delivery/czjl/C8USepRYFBSsSCL8HiYEuePFy6X5GwG5BulVEFmxYnqRE5QTA/audio.wav'}
https://replicate.delivery/yhqm/ITiK3WFTZxphM9tquShTocicckDXq5B2eEniBaczdSbsicoJA/out.mp3


In [18]:
def generate_suno_bark(prompt, api_token, text_temp=0.7, output_full=False, waveform_temp=0.7, history_prompt="announcer"):
    # Create a Replicate client instance with the API token
    client = replicate.Client(api_token=replicate_api_key)

    # Define the input parameters for the model
    input_params = {
        "prompt": prompt,
        "text_temp": text_temp,
        "output_full": output_full,
        "waveform_temp": waveform_temp,
        "history_prompt": "announcer",
    }

    # Run the model using Replicate API
    try:
        output = client.run(
            "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
            input=input_params
        )
        return output
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
# api_token = 'your_api_token_here'
# message = "Your prompt here"
output = generate_suno_bark(message, replicate_api_key)
print(output)

{'audio_out': 'https://replicate.delivery/czjl/q5wKXcw1c3ajHRjJz6rk1YzJeIWU3cU6Pf6HtaVSYOZoF5QTA/audio.wav'}


In [19]:
# Define your API token and prompt message
# api_token = 'your_api_token_here'
message = "The Waffle House messing it up for real with the pancakes and bacon and punk abstract jazz, yo!"

# Run the Epic Realism model
epicrealism_output = generate_epic_realism(message, replicate_api_key)
print("Epic Realism Output:")
print(epicrealism_output)

# Run the Meta MusicGen model
musicgen_output = generate_music_gen(message, replicate_api_key)
print("Meta MusicGen Output:")
print(musicgen_output)

# Run the Suno Bark model
bark_output = generate_suno_bark(message, replicate_api_key)
print("Suno Bark Output:")
print(bark_output)



Epic Realism Output:
['https://replicate.delivery/pbxt/ogO7Lgy5bHpuGp8TLp9TgLwuYrrt9hyUav8ltxlWhAmaRO0E/R8__00001_.webp']
Meta MusicGen Output:
https://replicate.delivery/yhqm/nJKhZpl5XK5GC5WbMXSmha8QPz1qreC4f5un2nLicYvxG5QTA/out.mp3
Suno Bark Output:
{'audio_out': 'https://replicate.delivery/czjl/27axINxSwiIpER6ofNkYMn5ywXGJdWoAohCs7fTe2oQWOyhmA/audio.wav'}


now define a little LLM utility function to help us out:

In [20]:
def get_llm_response(user_input):
    client = OpenAI(api_key=openai_api_key)
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
    {"role": "user", "content": user_input}
  ]
        )
    return response.choices[0].message.content

In [21]:
message = get_llm_response("short pirates sea shanty about mermaids")
print(message)
# Run the Suno Bark model
bark_output = generate_suno_bark(message, replicate_api_key)
print("Suno Bark Output:")
print(bark_output)

(Verse 1)
In the heart of the ocean, where the waves crash high,
There be a tale worth tellin', 'neath the starry sky.
Of fearless pirates sailin', on the open sea,
Where mermaids sing their melody, as wild and free.

(Chorus)
Yo-ho, yo-ho, a pirate's life we lead,
With mermaids in our sights, on the watery steed.
Their voices sweet as honey, their beauty all around,
We'll chase those mermaids true, till we've run aground.

(Verse 2)
Through storms and tempests, our ship sails on,
Seeking those mermaids, with their luring song.
Their tails a-glimmering, beneath the moonlit night,
We'll plow through the waters, searching with delight.

(Chorus)
Yo-ho, yo-ho, a pirate's life we lead,
With mermaids in our sights, on the watery steed.
Their voices sweet as honey, their beauty all around,
We'll chase those mermaids true, till we've run aground.

(Bridge)
But heed this warning, me hearties true,
For mermaids be deceivers, with eyes so blue.
With their siren's call, they'll lure you to your g

### Experiment: text to video

In [22]:
import replicate

message = "The Double RR diner messing it up for real with the pancakes and bacon and punk abstract jazz, yo!"

input = {
    "sampler": "klms",
    "max_frames": 100,
    "animation_prompts": message
}

output = client.run(
    "deforum/deforum_stable_diffusion:e22e77495f2fb83c34d5fae2ad8ab63c0a87b6b573b6208e1535b23b89ea66d6",
    input=input
)
print(output)
#=> "https://replicate.delivery/mgxm/873a1cc7-0427-4e8d-ab3c-...

https://replicate.delivery/yhqm/wx1JqbQ8Nrp9FRCHki5UAtzQZuUUqkvSOp54rNwVQx1rSO0E/out.mp4


In [24]:
message = "giddy up!!! [laughs] time for the thunderdome! [thunder]"

bark_output = generate_suno_bark(message, replicate_api_key)
print("Suno Bark Output:")
print(bark_output)

Suno Bark Output:
{'audio_out': 'https://replicate.delivery/czjl/wm5zfyqv3K3nSSmN3EgLHZVF9858HgTjQjf1xGHeFx32XyhmA/audio.wav'}


## Groq tests


In [25]:
! pip install groq

In [27]:
import os

from groq import Groq

client = Groq(
    api_key=groq_api_key)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial for several reasons:

**1. Real-Time Applications:**

* **Chatbots and Conversational AI:**  Speed is essential for natural-sounding conversations. Lag between user input and response makes interactions feel robotic and unnatural.
* **Live Transcription and Captioning:** Fast models can provide real-time transcriptions of speech, allowing for immediate accessibility and engagement in live events, meetings, or broadcasts.
* **Search Engines:** Faster response times lead to a better user experience, as users expect quick results when searching for information.

**2. Scalability and Efficiency:**

* **Processing Large Data Sets:**  Fast models can efficiently process massive amounts of text data, enabling large-scale analysis, summarization, and knowledge discovery.
* **Resource Efficiency:**  Faster training and inference processes consume less computational power and energy, making them more sustainable and cost-effective.

**3. Enhanced User Experience:

In [28]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "which model are you",
        }
    ],
    model="mixtral-8x7b-32768",
)

print(chat_completion.choices[0].message.content)

I am a transformer-based language model developed and trained by Mistral AI.
